# 농산물 도매 경매 데이터 처리 프로세스

## 1. 개요

본 문서는 2020년부터 2024년까지의 33개 공판장에서 수집된, 10개 선별 농산물 품목에 대한 도매 경매 데이터 처리 과정을 설명합니다. 이 처리된 데이터는 2025년 월별 농산물 시세 예측을 위한 기초 자료로 활용됩니다.

## 2. 데이터 개요

### 2.1 원본 데이터 형식
- 파일명 예시: "2020년 01월 공영도매시장 농협 공판장 거래금액 및 반입량-20200101_20200131"
- 총 60개 CSV 파일 (2020년 1월 ~ 2024년 12월, 월별 데이터)
- 주요 컬럼:
  - `auc_ymd`: 경매일자
  - `whsl_mrkt_code`: 도매시장코드
  - `whsl_mrkt_nm`: 도매시장명
  - `pdlt_code`: 품목코드
  - `pdlt_nm`: 품목명
  - `unit_qyt`: 단위수량
  - `prce`: 가격
  - `kg_unit_cnvr_qyt`: KG단위변환수량

## 3. 데이터 처리 과정

### 3.1 파일 로드 및 통합
- 정규식을 사용한 파일명 파싱으로 연도와 월 정보 추출
- 다중 인코딩 지원 (utf-8, cp949, euc-kr 등)
- 병렬 처리로 60개 파일 효율적 로드

```python
def extract_year_month(filename):
    """파일명에서 연도와 월을 추출합니다."""
    base_name = os.path.basename(filename)
    
    # 정규식을 사용하여 연도와 월을 추출
    year_month_pattern = r'(\d{4})년\s*(\d{2})월'
    match = re.search(year_month_pattern, base_name)
    
    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        return year, month
    else:
        # 파일명에서 날짜를 추출할 수 없는 경우, 하이픈 뒤의 날짜 범위에서 추출
        date_range_pattern = r'-(\d{4})(\d{2})(\d{2})_\d{4}\d{2}\d{2}'
        match = re.search(date_range_pattern, base_name)
        if match:
            year = int(match.group(1))
            month = int(match.group(2))
            return year, month
```

### 3.2 데이터 필터링 및 변환
- 10개 선별 농산물 품목만 필터링
- KG 단위로 가격 통일: `price_per_kg = prce / kg_unit_cnvr_qyt`
- 날짜 형식 표준화: `date = pd.to_datetime()`
- 데이터 유효성 검증: 음수 값, 0 값, NaN 값 처리

```python
# KG 단위로 데이터 변환
df_filtered['total_kg'] = df_filtered['kg_unit_cnvr_qyt'].apply(lambda x: max(x, 0) if pd.notnull(x) else 0)
df_filtered['total_sales'] = df_filtered['prce'].apply(lambda x: max(x, 0) if pd.notnull(x) else 0)

# price_per_kg 계산
df_filtered['price_per_kg'] = df_filtered.apply(
    lambda row: row['total_sales'] / row['total_kg'] if row['total_kg'] > 0 else np.nan, 
    axis=1
)
```

### 3.3 이상치 제거
- IQR(Inter-Quartile Range) 방식 적용
- 품목별, 월별로 그룹화하여 이상치 제거
- 데이터가 4개 이하인 경우 이상치 제거 생략

```python
def remove_outliers(group):
    if len(group) <= 4:  # 데이터가 너무 적으면 이상치 제거 생략
        return group
    q1 = group['price_per_kg'].quantile(0.25) # 1사분위수 (하위 25%)
    q3 = group['price_per_kg'].quantile(0.75) # 3사분위수 (상위 25%)
    iqr = q3 - q1 # 중간 50% 데이터를 의미
    lower_bound = q1 - 1.5 * iqr # 이상치 경계 하한선 
    upper_bound = q3 + 1.5 * iqr # 이상치 경계 상한선
    return group[(group['price_per_kg'] >= lower_bound) & (group['price_per_kg'] <= upper_bound)] # 이상치 제거
```

### 3.4 월별 가중 평균 계산
- 거래량을 가중치로 사용한 월별 평균 가격 계산
- 대량 거래 가격에 더 큰 가중치 부여
- 실제 시장 상황을 더 정확히 반영

```python
# 월별 가중 평균 계산 (거래량 기준)
monthly_data = (cleaned_data.groupby(['year', 'month', 'product']) # 연, 달, 품목으로 그룹화 
               .apply(lambda x: np.average(x['price_per_kg'], weights=x['total_kg']) # 람다 함수를 사용
               # 각 거래 총 키로그램 수를 가중치로 사용해 가중 평균을 낸다. 거래량이 많은 거래의 가격에 더 큰 비중
                    if x['total_kg'].sum() > 0 else x['price_per_kg'].mean())
                    # 만약 전체 kg 수가 0보다 많으면(거래량이 있으면) 가중 평균
                    # 없으면 모든 거래의 가격을 동일 비중으로 더하고 거래 횟수로 나눔(예외 처리, 데이터 누락이나 참조가격, 오류, 시세 정보 등을 다루기 위함함)
               .reset_index(name='weighted_avg_price')) # 계산 이후 컬럼명 변경
```

### 3.5 시계열 특성 생성
- 이전 달 가격 및 변화율
- 작년 동월 가격 및 변화율
- 3개월, 6개월, 12개월 이동평균
- 가격 변동성 (표준편차)

```python
for product in target_products:
    product_df = monthly_data[monthly_data['product'] == product].sort_values('date') # 품목 당 날짜에 맞추어 정렬
    
    if not product_df.empty:
        # 이전 달 가격, 변화율
        product_df['prev_month_price'] = product_df['weighted_avg_price'].shift(1) # 1달치 가격 연결
        product_df['monthly_change'] = product_df['weighted_avg_price'].pct_change() # 변화율 구하기
        
        # 작년 동월 가격, 변화율
        product_df['prev_year_price'] = product_df['weighted_avg_price'].shift(12) # 1년치(12개월) 가격 연결 
        product_df['yearly_change'] = product_df['weighted_avg_price'] / product_df['prev_year_price'] - 1 # 현재 가격을 작년 동월 가격으로 나누고 1을 빼서 변화율 계산 (현재 가격/작년 가격 > 1 상승, 1보다 작으면 하락, 1을 빼서 퍼센트 표현을 얻는다)
        
        # 이동평균 - 현재 행, 이전 N-1개 행의 평균 구하기
        product_df['ma_3'] = product_df['weighted_avg_price'].rolling(window=3).mean() # 3개월 이동 평균
        product_df['ma_6'] = product_df['weighted_avg_price'].rolling(window=6).mean() # 6개월 이동 평균
        product_df['ma_12'] = product_df['weighted_avg_price'].rolling(window=12).mean() # 12개월(1년) 이동 평균 
        
        # 가격 변동성
        product_df['volatility'] = product_df['weighted_avg_price'].rolling(window=3).std() # 3개월 표준편차 = 변동성 
```

## 4. 데이터 처리 결과

### 4.1 최종 데이터셋 구조
- 각 행은 특정 품목, 특정 월의 데이터를 나타냄
- 2020년 1월부터 2024년 12월까지 총 60개월 데이터
- 10개 선별 품목 각각에 대한 월별 시세 정보

### 4.2 주요 산출 컬럼
- `date`: 날짜 (연월)
- `product`: 품목명
- `weighted_avg_price`: 거래량 가중 평균 가격 (원/kg)
- `monthly_change`: 전월 대비 변화율 (%)
- `yearly_change`: 전년 동월 대비 변화율 (%)
- `ma_3`, `ma_6`, `ma_12`: 3, 6, 12개월 이동평균
- `volatility`: 가격 변동성 (3개월 표준편차)

## 5. 데이터 처리 시 고려사항

### 5.1 안전장치
- 중간 결과 정기 저장: 20개 파일마다 중간 결과 저장
- 파일 백업: 기존 결과 파일 자동 백업
- 예외 처리: 파일 읽기 오류, 계산 오류 등에 대한 처리

### 5.2 성능 최적화
- 병렬 처리: `ThreadPoolExecutor`를 사용한 다중 파일 동시 처리
- 메모리 효율성: 필요한 컬럼만 선택적 처리
- 진행 상황 모니터링: `tqdm`을 사용한 진행 상황 시각화

### 5.3 품질 보증
- 데이터 유효성 검증: 각 단계마다 데이터 품질 확인
- 품목별 통계 생성: 최소/최대/평균/표준편차 계산
- 결측치 처리: 계산 불가능한 값은 NaN으로 처리 후 분석 시 고려

## 6. 결론

이 데이터 처리 과정을 통해 2020년부터 2024년까지의 농산물 도매 경매 데이터를 분석하기 좋은 형태로 변환했습니다. 최종 데이터셋은 농산물 시세 예측 모델의 입력 데이터로 활용되며, 각 품목의 시간적 패턴과 관계를 분석하는 데 유용합니다.

In [3]:
import pandas as pd
import numpy as np
import os
from glob import glob
import concurrent.futures
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 1. 모든 CSV 파일 경로 가져오기
def get_all_csv_files(directory_path):
    """지정된 디렉토리에서 모든 CSV 파일 경로를 가져옵니다."""
    all_files = glob(os.path.join(directory_path, '*.csv'))
    return sorted(all_files)

In [5]:
# 2. 파일명에서 연월 추출하는 함수
def extract_year_month(filename):
    """파일명에서 연도와 월을 추출합니다."""
    # 파일명 예시: "2020년 01월 공영도매시장 농협 공판장 거래금액 및 반입량-20200101_20200131"
    base_name = os.path.basename(filename)
    
    # 정규식을 사용하여 연도와 월을 추출
    year_month_pattern = r'(\d{4})년\s*(\d{2})월'
    match = re.search(year_month_pattern, base_name)
    
    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        return year, month
    else:
        # 파일명에서 날짜를 추출할 수 없는 경우, 하이픈 뒤의 날짜 범위에서 추출
        date_range_pattern = r'-(\d{4})(\d{2})(\d{2})_\d{4}\d{2}\d{2}'
        match = re.search(date_range_pattern, base_name)
        if match:
            year = int(match.group(1))
            month = int(match.group(2))
            return year, month
        else:
            raise ValueError(f"파일명에서 연월을 추출할 수 없습니다: {base_name}")


In [6]:
# 3. 각 파일 처리 함수
def process_single_file(file_path, target_products):
    """단일 CSV 파일을 처리하는 함수"""
    try:
        # 파일에서 연월 추출
        year, month = extract_year_month(file_path)
        
        # 파일 로드
        df = pd.read_csv(file_path, encoding='euc-kr')
        
        # 날짜 형식 통일
        # auc_ymd가 문자열 형식인지 확인하고 적절히 처리
        if 'AUC_YMD' in df.columns:
            if df['AUC_YMD'].dtype == 'object':
                df['date'] = pd.to_datetime(df['AUC_YMD'], format='%Y%m%d', errors='coerce')
            else:  # 숫자형인 경우
                df['date'] = pd.to_datetime(df['AUC_YMD'].astype(str), format='%Y%m%d', errors='coerce')
        else:
            # auc_ymd 컬럼이 없는 경우, 파일명에서 추출한's 연월 정보로 날짜 생성
            df['date'] = pd.to_datetime(f'{year}-{month}-01')
        
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        
        # 10개 품목 필터링
        df_filtered = df[df['PDLT_NM'].isin(target_products)]
        
        # KG 단위로 데이터 변환
        # 값이 음수인 경우 처리
        df_filtered['total_kg'] = df_filtered['KG_UNIT_CNVR_QYT'].apply(lambda x: max(x, 0) if pd.notnull(x) else 0)
        df_filtered['total_sales'] = df_filtered['PRCE'].apply(lambda x: max(x, 0) if pd.notnull(x) else 0)
        
        # total_kg가 0인 경우 price_per_kg를 NaN으로 설정하여 나중에 제외
        df_filtered['price_per_kg'] = df_filtered.apply(
            lambda row: row['total_sales'] / row['total_kg'] if row['total_kg'] > 0 else np.nan, 
            axis=1
        )
        
        # 필요한 컬럼만 선택
        daily_data = df_filtered[['date', 'year', 'month', 'PDLT_NM', 'WMK_CORP_NM', 
                                 'total_kg', 'total_sales', 'price_per_kg']]
        
        # 컬럼명 변경
        daily_data = daily_data.rename(columns={'PDLT_NM': 'product', 'WMK_CORP_NM': 'NH-market'})
        
        # 유효한 가격 데이터만 선택 (0이나 NaN 제외)
        daily_data = daily_data[daily_data['price_per_kg'] > 0].dropna(subset=['price_per_kg'])
        
        print(f"처리 완료: {os.path.basename(file_path)}")
        return daily_data
    
    except Exception as e:
        print(f"파일 처리 중 오류 발생: {os.path.basename(file_path)}, 오류: {e}")
        return None


In [7]:
# 4. 모든 파일 처리 함수
def process_all_files(directory_path, target_products):
    """모든 CSV 파일을 병렬로 처리합니다."""
    all_files = get_all_csv_files(directory_path)
    print(f"총 {len(all_files)}개 파일을 처리합니다.")
    
    # 첫 번째 파일로 인코딩 테스트
    if all_files:
        try:
            # 인코딩 감지 (여러 일반적인 인코딩 시도)
            encodings = ['utf-8', 'cp949', 'euc-kr', 'latin1']
            detected_encoding = None
            
            for encoding in encodings:
                try:
                    with open(all_files[0], 'r', encoding=encoding) as f:
                        f.read(1024)  # 첫 1024바이트만 읽어서 테스트
                    detected_encoding = encoding
                    break
                except UnicodeDecodeError:
                    continue
            
            if detected_encoding:
                print(f"감지된 인코딩: {detected_encoding}")
            else:
                print("인코딩을 감지할 수 없습니다. 기본값 utf-8을 사용합니다.")
                detected_encoding = 'utf-8'
        except Exception as e:
            print(f"인코딩 감지 중 오류: {e}")
            detected_encoding = 'utf-8'
    else:
        print("처리할 파일이 없습니다.")
        return None
    
    # 다중 처리 설정
    max_workers = min(8, len(all_files))  # 최대 8개 스레드 또는 파일 수 중 작은 값
    all_data = []
    
    # ThreadPoolExecutor를 사용하여 병렬 처리
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 각 파일에 대해 process_single_file 함수 실행
        future_to_file = {executor.submit(process_single_file, file, target_products): file for file in all_files}
        
        for future in tqdm(concurrent.futures.as_completed(future_to_file), total=len(all_files)):
            file = future_to_file[future]
            try:
                data = future.result()
                if data is not None and not data.empty:
                    all_data.append(data)
                    
                    # 20개 파일마다 중간 결과 저장
                    if len(all_data) % 20 == 0:
                        temp_data = pd.concat(all_data, ignore_index=True)
                        temp_data.to_csv(f'temp_result_{len(all_data)}.csv', index=False)
                        print(f"중간 결과 저장: temp_result_{len(all_data)}.csv")
            except Exception as e:
                print(f"{os.path.basename(file)} 처리 중 예외 발생: {e}")
    
    # 모든 데이터 병합
    if all_data:
        combined_data = pd.concat(all_data, ignore_index=True)
        return combined_data
    else:
        print("처리된 데이터가 없습니다.")
        return None


In [8]:
# 5. 이상치 제거 함수
def remove_outliers(group):
    if len(group) <= 4:  # 데이터가 너무 적으면 이상치 제거 생략
        return group
    q1 = group['price_per_kg'].quantile(0.25)
    q3 = group['price_per_kg'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return group[(group['price_per_kg'] >= lower_bound) & (group['price_per_kg'] <= upper_bound)]


In [9]:

# 6. 메인 실행 함수
def main():
    # 설정
    data_directory = '2020-24 경매 데이터'  # CSV 파일이 있는 디렉토리 경로
    target_products = ['사과', '감귤', '수박', '배', '포도', '양파', '대파', '오이', '상추', '고구마']  # 분석할 10개 품목
    output_file = 'processed_monthly_agricultural_prices.csv'
    
    # 파일이 존재하는지 확인하고, 백업 생성
    if os.path.exists(output_file):
        backup_file = f"{output_file}.bak"
        try:
            os.rename(output_file, backup_file)
            print(f"기존 파일 백업: {backup_file}")
        except Exception as e:
            print(f"백업 생성 중 오류: {e}")
    
    # 모든 파일 처리
    combined_data = process_all_files(data_directory, target_products)
    
    if combined_data is not None:
        print(f"총 {len(combined_data)}개 데이터 로드 완료. 전처리 시작...")
        
        # 중간 결과 저장 (안전장치)
        temp_file = 'temp_processed_data.csv'
        combined_data.to_csv(temp_file, index=False)
        print(f"중간 데이터 저장 완료: {temp_file}")
        
        # 품목 및 날짜별로 그룹화하여 이상치 제거
        print("이상치 제거 중...")
        cleaned_data = combined_data.groupby(['year', 'month', 'product']).apply(remove_outliers).reset_index(drop=True)
        
        # 가격 범위 확인
        price_range = cleaned_data.groupby('product')['price_per_kg'].agg(['min', 'max', 'mean', 'std'])
        print("\n품목별 가격 범위:")
        print(price_range)
        
        # 월별 가중 평균 계산 (거래량 기준)
        print("월별 가중 평균 계산 중...")
        monthly_data = (cleaned_data.groupby(['year', 'month', 'product'])
                       .apply(lambda x: np.average(x['price_per_kg'], weights=x['total_kg']) if x['total_kg'].sum() > 0 else x['price_per_kg'].mean())
                       .reset_index(name='weighted_avg_price'))
        
        # 시계열 형식으로 날짜 변환
        monthly_data['date'] = pd.to_datetime(monthly_data['year'].astype(str) + '-' + 
                                            monthly_data['month'].astype(str) + '-01')
        
        # 추가 특성 생성
        print("추가 특성 생성 중...")
        final_data = []
        
        for product in target_products:
            product_df = monthly_data[monthly_data['product'] == product].sort_values('date')
            
            if not product_df.empty:
                # 기본 정보 확인
                print(f"\n{product} 데이터: {len(product_df)}개 월 (기간: {product_df['date'].min()} ~ {product_df['date'].max()})")
                
                # 이전 달 가격, 변화율
                product_df['prev_month_price'] = product_df['weighted_avg_price'].shift(1)
                product_df['monthly_change'] = product_df['weighted_avg_price'].pct_change()
                
                # 작년 동월 가격, 변화율
                product_df['prev_year_price'] = product_df['weighted_avg_price'].shift(12)
                product_df['yearly_change'] = product_df['weighted_avg_price'].div(product_df['prev_year_price']).sub(1)
                
                # 이동평균
                product_df['ma_3'] = product_df['weighted_avg_price'].rolling(window=3).mean()
                product_df['ma_6'] = product_df['weighted_avg_price'].rolling(window=6).mean()
                product_df['ma_12'] = product_df['weighted_avg_price'].rolling(window=12).mean()
                
                # 가격 변동성
                product_df['volatility'] = product_df['weighted_avg_price'].rolling(window=3).std()
                
                final_data.append(product_df)
            else:
                print(f"\n경고: {product}에 대한 데이터가 없습니다.")
        
        # 최종 데이터프레임
        result_df = pd.concat(final_data)
        
        # 저장
        result_df.to_csv(output_file, index=False)
        print(f"\n모든 처리가 완료되었습니다! 결과 파일: {output_file}")
        
        # 기본 통계 출력
        print("\n품목별 기본 통계:")
        summary = result_df.groupby('product')['weighted_avg_price'].describe()
        print(summary)
        
        return result_df
    else:
        print("처리할 데이터가 없습니다.")
        return None

In [10]:
if __name__ == "__main__":
    main()

총 59개 파일을 처리합니다.
감지된 인코딩: cp949


  2%|▏         | 1/59 [00:18<17:24, 18.00s/it]

처리 완료: 2020년 02월 공영도매시장 농협 공판장 거래금액 및 반입량-20200201_20200229.csv


  3%|▎         | 2/59 [00:18<07:27,  7.85s/it]

처리 완료: 2020년 01월 공영도매시장 농협 공판장 거래금액 및 반입량-20200101_20200131.csv


...

 34%|███▍      | 20/59 [04:35<34:25, 52.95s/it]

중간 결과 저장: temp_result_20.csv


100%|██████████| 59/59 [05:09<00:00,  5.24s/it]

중간 결과 저장: temp_result_40.csv


총 16326346개 데이터 로드 완료. 전처리 시작...
중간 데이터 저장 완료: temp_processed_data.csv
이상치 제거 중...

품목별 가격 범위:
               min           max         mean          std
product                                                   
감귤       54.545455  17886.666667  2458.938357  1910.428388
고구마       0.416667   8400.000000  1824.233698   936.391679
대파       50.000000   7814.285714  1787.941333   918.527622
배        33.333333  16640.000000  2695.215778  1417.287559
사과        5.555556  12500.000000  2950.066222  1713.833930
상추       10.483653  30911.000000  4369.938308  3537.310520
수박        7.142857   8875.000000  1687.319207   843.483748
양파        7.636364   2875.000000   949.067262   413.882544
오이        0.100000   9466.666667  1927.593139  1178.994327
포도       11.627907  27280.000000  5805.184108  2952.418444
월별 가중 평균 계산 중...
추가 특성 생성 중...

사과 데이터: 59개 월 (기간: 2020-01-01 00:00:00 ~ 2024-12-01 00:00:00)

감귤 데이터: 59개 월 (기간: 2020-01-01 00:00:00 ~ 2024-12-01 00:00:00)

수박 데이터: 59개 월 (기간: 2020-01-01 00:00:00 